# Create Table from instance segmentation polygons

This notebook demonstrates how to create a table of instance segmentation polygons.

![img](../images/instance-segmentation.jpg)

<!-- Tags: ["Instance Segmentation", "COCO", "Write Table", "Polygons"] -->

"Instance segmentation", as opposed to "semantic segmentation", refers to situations where each "object" in your dataset has some associated segmentation mask (in this case described by a polygon) which might overlap with other objects. "Semantic segmentation", by contrast, refers to situations where each pixel in each image of your dataset is associated with exactly one class label. See [Import semantic segmentation dataset](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/1-create-tables/create-semantic-segmentation-dataset.ipynb) for an example of how to create a table from semantic segmentation masks.

This notebook will use instance segmentations represented as a list of polygons. For an example of how to use instance segmentations represented as per-pixel masks, see [this notebook](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/1-create-tables/create-instance-segmentation-masks-table.ipynb).

We will use the COCO128 for this example. Note that while 3LC has built-in functionality to facilitate the loading of COCO-style datasets, this example will more "manually" load the dataset to demonstrate how this can be done in general.

## Project Setup

In [1]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "COCO128 Segmentation Polygons"
DATA_PATH = "../../data"

## Install dependencies

In [2]:
%pip install 3lc

/Users/gudbrand/Projects/3lc-examples/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


## Imports

In [3]:
import json
from pathlib import Path

import tlc

## Get images and polygons

In [4]:
coco128_path = Path(DATA_PATH) / "coco128"

# Open COCO128 annotations.json file and load the annotations
with open(coco128_path / "annotations.json") as f:
    annotations = json.load(f)

# Get the images, segmentations and classes from the data file
images = annotations["images"]
instances = annotations["annotations"]
classes = [category["name"] for category in annotations["categories"]]

We want to create a list of rows with the following structure:

```
{
    "image": str,
    "segmentation: {
        "image_width": int,
        "image_height": int,
        "instance_properties": {
            "label": list[int],
        },
        "polygons": list[list[float]]
    }
}
```

Where `image` is a path to the image file, and `segmentation` is a dict of the format expected by 3LC for polygon-based instance segmentation. Its fields are:

- `image_width`: The width of the image
- `image_height`: The height of the image
- `instance_properties`: A dict of properties for each instance in the segmentation. The keys are the names of the properties, and the values are a list containing the value of that property for each instance in the image. Examples of instance properties include `label` and `confidence`.
- `polygons`: A list of lists, one for each instance in the image. Each inner list, representing a single polygon, contains alternating x and y coordinates of the vertices of the polygon. E.g. `[[x1, y1, x2, y2, ...], [x1, y1, x2, y2, ...], ...]`. These coordinates can be absolute, in terms of pixels, or relative, in terms of the width and height of the image.

We will now create a list of rows with the above structure.

In [5]:
# Create a list of rows, holding information about each image
# The instance properties are initially empty, and will be populated later
rows = [
    {
        "image": (coco128_path / "images" / image["file_name"]).absolute().as_posix(),
        "segmentation": {
            "image_width": image["width"],
            "image_height": image["height"],
            "instance_properties": {
                "label": [],
            },
            "polygons": [],
        },
    }
    for image in images
]

In [ ]:
for instance in instances:
    # Get the row index for the instance
    row_id = instance["image_id"]

    # Get the label and polygon for the instance
    label = instance["category_id"]

    if isinstance(instance["segmentation"], list):
        polygon = instance["segmentation"][0]
    else:
        continue
    
    # Add the label and polygon to the row
    rows[row_id]["segmentation"]["instance_properties"]["label"].append(label)
    rows[row_id]["segmentation"]["polygons"].append(polygon)

## Create a table from the rows

We will now use a `TableWriter` to write the rows to a `Table`.

In [ ]:
# Define the column schemas of our Table
column_schemas = {
    "image": tlc.ImageUrlSchema(),
    "segmentation": tlc.SegmentationSchema(
        label_value_map={i: tlc.MapElement(c) for i, c in enumerate(classes)},
        sample_type=tlc.InstanceSegmentationPolygons.sample_type,
    ),
}

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


AttributeError: module 'tlc' has no attribute 'SegmentationPolygons'

In [ ]:
# Create a TableWriter
table_writer = tlc.TableWriter(
    table_name="initial",
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas=column_schemas,
    if_exists="rename",
)

In [ ]:
# Write the rows to the Table
for row in rows:
    table_writer.add_row(row)

# Once we are done, call `finalize()` to create the Table
table = table_writer.finalize()

The table will now be visible in the 3LC Dashboard and can be used in any of your scripts or notebooks.